In [2]:
!pip install accelerate
!pip install transformers
!pip install torch
!pip install pandas
!pip install requests
!pip install langchain
!pip install faiss-cpu
!pip install rank_bm25
!pip install langchain_huggingface
!pip install -U transformers accelerate peft
!pip install -U bitsandbytes
!pip install langchain_community


  Using cached langchain_community-0.3.27-py3-none-any.whl.metadata (2.9 kB)
  Using cached aiohttp-3.12.15-cp313-cp313-macosx_11_0_arm64.whl.metadata (7.7 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached pydantic_settings-2.10.1-py3-none-any.whl.metadata (3.4 kB)
  Using cached httpx_sse-0.4.1-py3-none-any.whl.metadata (9.4 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.4.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached frozenlist-1.7.0-cp313-cp313-macosx_11_0_arm64.whl.metadata (18 kB)
  Using cached multidict-6.6.4-cp313-cp313-macosx_11_0_arm64.whl.metadata (5.3 kB)
  Using cached propcache-0.3.2-cp313-cp313-macosx_11_0_arm64.whl.metadata (12 kB)
  Using cached yarl-1.20.1-cp313-cp313-macosx_11_0_arm64.whl.metadata (73 kB)
  Using cached marshmallow-3.26.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached typing_inspect-0

In [3]:
import re
import os
import pandas as pd
from tqdm import tqdm
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_community.embeddings import HuggingFaceEmbeddings
from sentence_transformers import CrossEncoder
from langchain_huggingface import HuggingFacePipeline
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough

/opt/anaconda3/envs/financeAI/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
test = pd.read_csv('/Users/seojeonghun/Desktop/금융 AI 챌린지/data/test.csv')

In [ ]:
# 객관식 여부 판단 함수
def is_multiple_choice(question_text):
    """
    객관식 여부를 판단: 2개 이상의 숫자 선택지가 줄 단위로 존재할 경우 객관식으로 간주
    """
    lines = question_text.strip().split("\n")
    option_count = sum(bool(re.match(r"^\s*[1-9][0-9]?\s", line)) for line in lines)
    return option_count >= 2


# 질문과 선택지 분리 함수
def extract_question_and_choices(full_text):
    """
    전체 질문 문자열에서 질문 본문과 선택지 리스트를 분리
    """
    lines = full_text.strip().split("\n")
    q_lines = []
    options = []

    for line in lines:
        if re.match(r"^\s*[1-9][0-9]?\s", line):
            options.append(line.strip())
        else:
            q_lines.append(line.strip())

    question = " ".join(q_lines)
    return question, options

In [ ]:
# 프롬프트 생성기
def make_prompt_auto(text):
    if is_multiple_choice(text):
        question, options = extract_question_and_choices(text)
        prompt = (
                "당신은 금융보안 전문가입니다.\n"
                "아래 질문에 대해 적절한 **정답 선택지 번호만 출력**하세요.\n\n"
                f"질문: {question}\n"
                "선택지:\n"
                f"{chr(10).join(options)}\n\n"
                "답변:"
                )
    else:
        prompt = (
                "당신은 금융보안 전문가입니다.\n"
                "아래 주관식 질문에 대해 정확하고 간략한 설명을 작성하세요.\n\n"
                f"질문: {text}\n\n"
                "답변:"
                )
    return prompt

### Model Load

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [ ]:
# 단계 1: 문서 로드(Load Documents)
# pdf가 아니라 word이면 여기서 loader을 변경하면 된다.

loader = PyMuPDFLoader("/content/total_law.pdf")
docs = loader.load()
print(f"문서의 페이지수: {len(docs)}")

# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)

print(len(split_documents))

문서의 페이지수: 660
1639


ollama 임베딩 모델 쓰는 방법(모델이 무거워서 진행 못함)

In [ ]:
# ollama download
!curl -fsSL https://ollama.com/install.sh | sh
!nohup ollama serve > /dev/null 2>&1 &

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
                                                                           1.3%^C


In [ ]:
!ollama pull mxbai-embed-large

In [ ]:
# 단계 3 : 임베딩
from langchain_community.embeddings import OllamaEmbeddings

ollama_embeddings = OllamaEmbeddings(
    model = 'mxbai-embed-large'
)

/tmp/ipython-input-3257686663.py:4: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  ollama_embeddings = OllamaEmbeddings(


In [ ]:
# 단계 4: DB 생성(Create DB) 및 저장
vectorstore = FAISS.from_documents(documents=split_documents, embedding=ollama_embeddings)

KeyboardInterrupt: 

e5 모델 임베딩

In [ ]:
# 한국어/멀티: e5-multilingual, bge-m3, MiniLM 등
emb = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-small")   # 빠름

# 단계 4: DB 생성 및 저장
vectorstore = FAISS.from_documents(split_documents, emb)

/tmp/ipython-input-136386298.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  emb = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-small")   # 빠름
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.

### 단계 5. retreiver

In [ ]:
# 단계 5-1 : ensemble REtriever 생성
from langchain.retrievers import EnsembleRetriever

# dense retreiver
dense_retriever = vectorstore.as_retriever(
    search_kwargs={"k":20}
)

# sparse retriever
from langchain_community.retrievers import BM25Retriever
bm_25_retriever = BM25Retriever.from_documents(split_documents)
bm_25_retriever.k = 20

# (선택) 청크에 고유 ID가 있다면 중복 판정 키 지정
# for i, d in enumerate(split_documents): d.metadata["chunk_id"] = i
ensemble = EnsembleRetriever(
    retrievers=[dense_retriever, bm_25_retriever],
    weights=[0.6, 0.4],
    # id_key="chunk_id",  # 위처럼 넣었다면 중복 제거 기준으로 추천
    # c=60,               # RRF 파라미터(기본 60)
)

# docs = ensemble.get_relevant_documents("개인정보보호법 제57조에 따르면, 단체소송의 절차에 관하여 필요한 사항은 무엇에 의해 정해지는가?")

In [ ]:
retriever = ensemble

In [ ]:
# test 해보기

query = "개인정보보호법 제57조에 따르면, 단체소송의 절차에 관하여 필요한 사항은 무엇에 의해 정해지는가?"
# 권장: get_relevant_documents 대신 invoke 사용 (LCEL 표준)
docs = ensemble.invoke(query)

# 최종 개수를 10개로 제한하고 싶다면 여기서 잘라주세요
docs = docs[:10]

### Reranking

In [ ]:
# 1차 Reranker (빠른 모델)
reranker_fast = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-12-v2')

# 2차 Reranker (정확도 높은 모델, 예시로 다른 모델 사용 가능)
reranker_slow = CrossEncoder('cross-encoder/ms-marco-electra-base')

def build_multi_reranked_context(question: str) -> str:
    """
    Multi-step reranking:
    1. Retriever → 상위 100개
    2. Reranker_fast → 상위 10개
    3. Reranker_slow → 최종 순위 결정 → 컨텍스트 구성
    """
    # 1) Retriever에서 후보 문서 수집
    docs = retriever.get_relevant_documents(question)
    if not docs:
        return ""

    # ---------- 1차 rerank ----------
    # Retriever 결과 중 상위 100개만 사용
    top_100 = docs[:10]
    pairs_1 = [(question, d.page_content) for d in top_100]
    scores_1 = reranker_fast.predict(pairs_1)

    ranked_1 = sorted(zip(top_100, scores_1), key=lambda x: x[1], reverse=True)
    top_5_stage1 = [d for d, _ in ranked_1[:5]]

    # ---------- 2차 rerank ----------
    pairs_2 = [(question, d.page_content) for d in top_10_stage1]
    scores_2 = reranker_slow.predict(pairs_2)

    ranked_2 = sorted(zip(top_10_stage1, scores_2), key=lambda x: x[1], reverse=True)
    final_docs = [d for d, _ in ranked_2]

    # ---------- 최종 컨텍스트 ----------
    context = "\n\n---\n\n".join(d.page_content for d in final_docs)
    return context

### 모델 불러오기

In [ ]:
model_name = "beomi/gemma-ko-7b"

# Tokenizer 및 모델 로드 (4bit)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    load_in_4bit=True,
    torch_dtype=torch.float16
)

# Inference pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto"
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

model-00005-of-00006.safetensors:   0%|          | 0.00/2.92G [00:00<?, ?B/s]

model-00006-of-00006.safetensors:   0%|          | 0.00/2.37G [00:00<?, ?B/s]

...

model-00003-of-00006.safetensors:   0%|          | 0.00/2.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

Device set to use cuda:0


NameError: name 'HuggingFacePipeline' is not defined

In [ ]:
# llm으로 감싸기
llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
# 프롬프트 빌더: 문자열을 리턴 (템플릿 객체 X)
def build_prompt(inputs: dict) -> str:
    text = inputs["question"]
    context = inputs["context"]
    if is_multiple_choice(text):
        question, options = extract_question_and_choices(text)
        return (
            "당신은 금융보안 전문가입니다.\n"
            "아래 질문에 대해 적절한 **정답 선택지 번호만 출력**하세요.\n\n"
            f"질문: {question}\n"
            "선택지:\n"
            f"{chr(10).join(options)}\n\n"
            f"참고: {context}\n"
            "답변:"
        )
    else:
        return (
            "당신은 금융보안 전문가입니다.\n"
            "아래 주관식 질문에 대해 정확하고 간략한 설명을 작성하세요.\n\n"
            f"질문: {text}\n\n"
            f"참고: {context}\n"
            "답변:"
        )

In [ ]:
# LCEL 체인 구성
chain = (
    {
        "context": RunnableLambda(build_multi_reranked_context),  # RAG
        "question": RunnablePassthrough(),                       # 원문 질문
    }
    | RunnableLambda(build_prompt)   # 문자열 프롬프트 생성
    | llm                            # HF pipeline 호출
    | StrOutputParser()              # 문자열만 추출
)


In [ ]:
# 후처리 함수
def extract_answer_only(generated_text: str, original_question: str) -> str:
    """
    - "답변:" 이후 텍스트만 추출
    - 객관식 문제면: 정답 숫자만 추출 (실패 시 전체 텍스트 또는 기본값 반환)
    - 주관식 문제면: 전체 텍스트 그대로 반환
    - 공백 또는 빈 응답 방지: 최소 "미응답" 반환
    """
    # "답변:" 기준으로 텍스트 분리
    if "답변:" in generated_text:
        text = generated_text.split("답변:")[-1].strip()
    else:
        text = generated_text.strip()

    # 공백 또는 빈 문자열일 경우 기본값 지정
    if not text:
        return "미응답"

    # 객관식 여부 판단
    is_mc = is_multiple_choice(original_question)

    if is_mc:
        # 숫자만 추출
        match = re.match(r"\D*([1-9][0-9]?)", text)
        if match:
            return match.group(1)
        else:
            # 숫자 추출 실패 시 "0" 반환
            return "0"
    else:
        return text


### Test Inference

In [ ]:
df = pd.read_csv('/content/test.csv')
test = df.copy()
# test = df.head(50).copy() # 일부만 test할 때 쓰기

In [ ]:
preds = []

for q in tqdm(test["Question"], desc="Inference"):
    # 체인 실행 → 최종 문자열 반환
    resp_text = chain.invoke(q)

    # 후처리로 답변만 추출
    pred_answer = extract_answer_only(resp_text, original_question=q)
    preds.append(pred_answer)

Inference:   0%|          | 0/50 [00:00<?, ?it/s]


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
preds = []

for q in tqdm(test['Question'], desc="Inference"):
    response = chain.invoke(q)

    output = pipe(prompt, max_new_tokens=128, temperature=0.2, top_p=0.9)
    pred_answer = extract_answer_only(response[0]["generated_text"], original_question=q)
    preds.append(pred_answer)

### submission

In [ ]:
sample_submission = pd.read_csv('./sample_submission.csv')
sample_submission['Answer'] = preds
sample_submission.to_csv('./baseline_submission.csv', index=False, encoding='utf-8-sig')